In [ ]:
import pandas as pd
import re
import urllib.parse
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

In [ ]:
df=pd.read_csv("scrap_leads.csv")
df.columns=df.columns.str.strip()

In [ ]:
def process_leads(df):
    # Deduplication
    df = df.drop_duplicates(subset=['Company', 'Owner Email'])

    # Email validation
    def is_valid_email(email):
        if pd.isna(email):
            return False
        regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
        return re.match(regex, email) is not None

    df['Valid Email'] = df['Owner Email'].apply(is_valid_email)

    # Free domain detection
    free_domains = ['gmail.com', 'yahoo.com', 'hotmail.com', 'outlook.com']
    def is_free_email(email):
        if pd.isna(email) or not is_valid_email(email):
            return False
        domain = email.split('@')[-1]
        return domain in free_domains

    df['Free Email'] = df['Owner Email'].apply(is_free_email)

    # Missing fields
    df['Missing Website'] = df['Website'].isna() | (df['Website'] == '')
    df['Missing Phone'] = df['Owner Phone'].isna() | (df['Owner Phone'] == '')

    # LinkedIn URL generation
    def generate_linkedin_search_url(company_name):
        if pd.isna(company_name) or company_name.strip() == '':
            return ''
        query = f"{company_name} site:linkedin.com/company"
        return f"https://www.google.com/search?q={urllib.parse.quote_plus(query)}"

    linkedin_column = 'Owner LinkedIn'
    def fill_missing_linkedin(row):
        if pd.isna(row[linkedin_column]) or row[linkedin_column].strip() == '':
            return generate_linkedin_search_url(row['Company'])
        return row[linkedin_column]

    df[linkedin_column] = df.apply(fill_missing_linkedin, axis=1)

    # Scoring
    def revenue_score(revenue):
        try:
            num = int(str(revenue).replace('Cr', '').replace(',', '').strip())
            return 2 if num >= 5000 else 0
        except:
            return 0

    def employee_score(count):
        try:
            return 2 if int(count) > 10000 else 0
        except:
            return 0

    def lead_score(row):
        score = 0
        if row['Valid Email']:
            score += 2
        if not row['Free Email'] and row['Valid Email']:
            score += 2
        score += revenue_score(row['Revenue'])
        score += employee_score(row['Employees Count'])
        if row['Missing Website']:
            score -= 1
        if row['Missing Phone']:
            score -= 1
        return score

    df['Lead Score'] = df.apply(lead_score, axis=1)

    def categorize(score):
        if score > 6:
            return 'High'
        elif score >= 3:
            return 'Medium'
        else:
            return 'Low'

    df['Lead Priority'] = df['Lead Score'].apply(categorize)

    return df

In [ ]:
cleaned_df = process_leads(df)


/tmp/ipython-input-4-2737168925.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Valid Email'] = df['Owner Email'].apply(is_valid_email)
/tmp/ipython-input-4-2737168925.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Free Email'] = df['Owner Email'].apply(is_free_email)
/tmp/ipython-input-4-2737168925.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [ ]:
cleaned_df.head(8)

,Company,Website,Industry,Revenue,Employees Count,Owner Email,Owner Phone,Owner LinkedIn,Valid Email,Free Email,Missing Website,Missing Phone,Lead Score,Lead Priority
0,Tata Consultancy Services,www.tcs.com,IT Services,50000Cr,400000,rajiv.sharma@tcs.com,9876543210,https://www.google.com/search?q=Tata+Consultan...,True,False,False,False,8,High
1,Caprae Capital,www.caprae.com,IT Services,100000Cr,820000,kevinhong@yahoo.com,6578943521,linkedin.com/in/kevinhshong,True,True,False,False,6,Medium
2,Reliance Industries,www.ril.com,Oil & Gas,300000Cr,250000,mukesh@ril.com,9123456789,linkedin.com/in/mukeshambani,True,False,False,False,8,High
3,Infosys,www.infosys.com,IT Services,80000Cr,250000,sundar.kumar@infosys.com,9988776655,linkedin.com/in/sundarkumar,True,False,False,False,8,High
4,Flipkart,www.flipkart.com,E-commerce,35000Cr,30000,rajat@gmail.com,9876543211,linkedin.com/in/rajatverma,True,True,False,False,6,Medium
5,Zomato,www.zomato.com,Food Delivery,4000Cr,5000,deepak@invalid,9888776655,linkedin.com/in/deepakjoshi,False,False,False,False,0,Low


In [ ]:
Markdown("## 🧠 Lead Scoring Dashboard\nClean, score, and prioritize your leads.")

# Summary
Markdown(f"""
- **Total Leads:** {len(cleaned_df)}
- **Valid Business Emails:** {cleaned_df['Valid Email'].sum()}
- **High Priority Leads:** {sum(cleaned_df['Lead Priority'] == 'High')}
""")


- **Total Leads:** 6  
- **Valid Business Emails:** 5  
- **High Priority Leads:** 3


In [ ]:
priority_filter = widgets.Dropdown(options=['All', 'High', 'Medium', 'Low'], description='Priority:')
display(priority_filter)

def show_filtered(priority_level):
    if priority_level == 'All':
        display(cleaned_df[['Company', 'Owner Email', 'Lead Score', 'Lead Priority', 'Website', 'Owner Phone', 'Owner LinkedIn']])
    else:
        filtered = cleaned_df[cleaned_df['Lead Priority'] == priority_level]
        display(filtered[['Company', 'Owner Email', 'Lead Score', 'Lead Priority', 'Website', 'Owner Phone', 'Owner LinkedIn']])

priority_filter.observe(lambda change: show_filtered(change.new), names='value')

Dropdown(description='Priority:', options=('All', 'High', 'Medium', 'Low'), value='All')

In [ ]:
def highlight_priority(val):
    color = {'High': 'lightgreen', 'Medium': 'yellow', 'Low': 'red'}.get(val, '')
    return f'background-color: {color}'

In [ ]:
styled = cleaned_df.style.applymap(highlight_priority, subset=['Lead Priority'])
styled

/tmp/ipython-input-18-3764039018.py:1: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled = cleaned_df.style.applymap(highlight_priority, subset=['Lead Priority'])


,Company,Website,Industry,Revenue,Employees Count,Owner Email,Owner Phone,Owner LinkedIn,Valid Email,Free Email,Missing Website,Missing Phone,Lead Score,Lead Priority
0,Tata Consultancy Services,www.tcs.com,IT Services,50000Cr,400000,rajiv.sharma@tcs.com,9876543210,https://www.google.com/search?q=Tata+Consultancy+Services+site%3Alinkedin.com%2Fcompany,True,False,False,False,8,High
1,Caprae Capital,www.caprae.com,IT Services,100000Cr,820000,kevinhong@yahoo.com,6578943521,linkedin.com/in/kevinhshong,True,True,False,False,6,Medium
2,Reliance Industries,www.ril.com,Oil & Gas,300000Cr,250000,mukesh@ril.com,9123456789,linkedin.com/in/mukeshambani,True,False,False,False,8,High
3,Infosys,www.infosys.com,IT Services,80000Cr,250000,sundar.kumar@infosys.com,9988776655,linkedin.com/in/sundarkumar,True,False,False,False,8,High
4,Flipkart,www.flipkart.com,E-commerce,35000Cr,30000,rajat@gmail.com,9876543211,linkedin.com/in/rajatverma,True,True,False,False,6,Medium
5,Zomato,www.zomato.com,Food Delivery,4000Cr,5000,deepak@invalid,9888776655,linkedin.com/in/deepakjoshi,False,False,False,False,0,Low


In [ ]:
cleaned_df.to_csv("cleaned_df.csv", index=False)
print("Cleaned leads saved as 'cleaned_df.csv'")

Cleaned leads saved as 'cleaned_df.csv'
